In [5]:
#!import config/Settings.cs

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc3

In [2]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [6]:
using Kernel = Microsoft.SemanticKernel.Kernel;

var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.WithAzureOpenAIChatCompletionService(model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

builder.AddAzureOpenAIChatCompletion(model, model, azureEndpoint, apiKey, serviceId: orgId);

var kernel = builder.Build();

# 2.1. 🚀 Semantic Kernel Pipeline: Calling Chain of Inline Semantic Functions

In the SK pipeline, you can create a powerful and expressive chain of inline functions. These functions, when executed sequentially, form a cohesive and efficient processing pipeline. Each function plays a specific role in transforming and analyzing data, contributing to the overall semantic understanding of the code.


In [23]:
using System;
using Microsoft.SemanticKernel.Orchestration;

SKContext context = kernel.CreateNewContext();

In [24]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var requestSettings = new OpenAIRequestSettings
{
    Temperature = 0.7,
    ServiceId = gpt4ServiceId,
};

context.Variables["NumberOfParticipants"] = "3";
context.Variables["NumberOfWords"] = "25";

var chatStoryPrompt = "Craft a {{$NumberOfWords}}-word chat transcript involving {{$NumberOfParticipants}} users engaging with an online virtual assistant.";
var chatStoryFunction = kernel.CreateSemanticFunction(
    promptTemplate: chatStoryPrompt, 
    pluginName: nameof(chatStoryPrompt),
    requestSettings: requestSettings);

In [25]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

var requestSettings = new OpenAIRequestSettings
{
    Temperature = 2,
    ServiceId = gpt4ServiceId,
};

context.Variables["Lang"] = "Italian";

var translateStoryPrompt = "You are the best linguist. Please translate the given TEXT to {{$Lang}}. TEXT: {{$Input}}";

var translateStoryFunction = kernel.CreateSemanticFunction(
    promptTemplate: translateStoryPrompt, 
    pluginName: nameof(translateStoryPrompt),
    requestSettings: requestSettings);

In [26]:
ISKFunction[] pipeline = new ISKFunction[]
{
    chatStoryFunction,
    translateStoryFunction,
};

var pipelineResult = await kernel.RunAsync(context.Variables, pipeline);

Console.WriteLine($"RESULT: {string.Join("\n\n===================\n\n", pipelineResult.FunctionResults.Select(_ => _.GetValue<string>()))}");

RESULT: User1: "Hello, Assistant."
Assistant: "Hi, how can I help you?"
User2: "What's the weather like?"
Assistant: "Let me check that for you."
User3: "Thanks, Assistant."


User1: "Ciao, Assistente."
Assistente: "Ciao, come posso aiutarti?"
User2: "Com'è il tempo?"
Assistente: "Lascia che lo controlli per te."
User3: "Grazie, Assistente."


# 2.2 🚀 Using Built-in Native Semantic Kernel Plugins: Loading Functions

The SK provides a set of built-in native plugins that offer powerful functionality for code analysis and manipulation. Loading functions from these plugins allows you to leverage their capabilities seamlessly within your code. Follow the steps below to harness the features of built-in native SK plugins.


In [27]:
using Microsoft.SemanticKernel.Plugins.Core;

var timeFunctions = kernel.ImportFunctions(new TimePlugin(), "time");
context.Variables["input"] = "100";
var result = await kernel.RunAsync(context.Variables, timeFunctions["daysAgo"]);

Console.WriteLine(result);

Friday, September 1, 2023


In [28]:
using System.Threading.Tasks;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;
using Microsoft.SemanticKernel.Orchestration;
using Microsoft.SemanticKernel.TemplateEngine;
using Microsoft.SemanticKernel.TemplateEngine.Basic;
using Microsoft.DotNet.Interactive;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;
using System.ComponentModel;

public class CustomPlugin
{
    private readonly IKernel _kernel;
    private readonly IPromptTemplateFactory _promptTemplateFactory;
    private readonly string _serviceId;

    public CustomPlugin(IKernel kernel, IPromptTemplateFactory promptTemplateFactory = null, string serviceId = "gpt35TurboServiceId")
    {
        _kernel = kernel;
        _promptTemplateFactory = promptTemplateFactory ?? new BasicPromptTemplateFactory();
        _serviceId = serviceId;
    }

    [SKFunction, Description("Initiates a chat with the ChatBot.")]
    public async Task<string> ChatAsync(SKContext context)
    {
        var prompt = @"
        Today is: {{time.Date}}
        Current time is: {{time.Time}}

        ChatBot can have a conversation with you about any topic.
            It can give detailed answer or say 'I don't know' if it does not have an answer.

            {{$history}}
            User: {{$message}}
            ChatBot:";

        var renderedPrompt = await _promptTemplateFactory.Create(prompt, new PromptTemplateConfig()).RenderAsync(context);

        var skFunction = _kernel.CreateSemanticFunction(
            promptTemplate: renderedPrompt,
            functionName: nameof(ChatAsync),
            pluginName: nameof(CustomPlugin),
            description: "Complete the prompt.");

        var requestSettings = new OpenAIRequestSettings
        {
            Temperature = 0.7,
            ServiceId = _serviceId
        };

        var result = await skFunction.InvokeAsync(context: context, requestSettings: requestSettings);
        var resultAsString = result.GetValue<string>();
        // Append the new interaction to the chat history
        string history = context.Variables["history"];
        history += $"\nUser: {context.Variables["message"]}\nSuggestions: {resultAsString}\n";
        return history;
    }

    [SKFunction, Description("Initiates a translation task with the ChatBot by requesting the translation of a given text to a specified language.")]
    public async Task<string> TranslateAsync(SKContext context)
    {
        var prompt = @"You are the best linguist. Please translate the given TEXT to {{$lang}}. TEXT: {{$Input}}";

        var renderedPrompt = await _promptTemplateFactory.Create(prompt, new PromptTemplateConfig()).RenderAsync(context);

        var skFunction = _kernel.CreateSemanticFunction(
            promptTemplate: renderedPrompt,
            functionName: nameof(TranslateAsync),
            pluginName: nameof(CustomPlugin),
            description: "Complete the prompt.");

        var requestSettings = new OpenAIRequestSettings
        {
            Temperature = 0.1,
            ServiceId = _serviceId
        };

        var result = await skFunction.InvokeAsync(context: context, requestSettings: requestSettings);
        return result.GetValue<string>();
    }
}

In [29]:
var functions = kernel.ImportFunctions(new CustomPlugin(kernel, serviceId: gpt4ServiceId));
Console.WriteLine($"{string.Join("\n", functions.Select(_ => $"[{_.Value.Name}] : {_.Value.Description}"))}");

[Chat] : Initiates a chat with the ChatBot.
[Translate] : Initiates a translation task with the ChatBot by requesting the translation of a given text to a specified language.


In [30]:
context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Italian";

var result = await kernel.RunAsync(functions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Russian";

result = await kernel.RunAsync(functions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

context.Variables["input"] = "Translate me";
context.Variables["lang"] = "Chinese";

result = await kernel.RunAsync(functions["Translate"], context.Variables);

Console.WriteLine($"{result.GetValue<string>()}");

Traducimi
Переведите меня
翻译我


In [31]:
var pipeline = new []
{
    functions["Chat"],
    functions["Translate"]
};

context.Variables["lang"] = "Russian";
context.Variables["message"] = "Hello";
context.Variables["history"] = string.Empty;

var result = await kernel.RunAsync(context.Variables, pipeline);
var resultString = result.GetValue<string>();

Console.WriteLine(context.Variables["history"]);

Error: (2,32): error CS0234: The type or namespace name 'Orchestration' does not exist in the namespace 'Microsoft.SemanticKernel' (are you missing an assembly reference?)
(4,1): error CS0246: The type or namespace name 'SKContext' could not be found (are you missing a using directive or an assembly reference?)
(4,28): error CS1061: 'Kernel' does not contain a definition for 'CreateNewContext' and no accessible extension method 'CreateNewContext' accepting a first argument of type 'Kernel' could be found (are you missing a using directive or an assembly reference?)

In [32]:
Func<string, Task> Chat = async (string input) => {
    // Save new message in the context variables
    context.Variables["message"] = input;
    var answer = await kernel.RunAsync(context.Variables, pipeline);
};

Error: (3,27): error CS0246: The type or namespace name 'OpenAIRequestSettings' could not be found (are you missing a using directive or an assembly reference?)
(10,37): error CS1061: 'Kernel' does not contain a definition for 'CreateSemanticFunction' and no accessible extension method 'CreateSemanticFunction' accepting a first argument of type 'Kernel' could be found (are you missing a using directive or an assembly reference?)

In [33]:
await Chat("Favorite color?");
await Chat("Best movie genre?");
await Chat("Morning or night?");

Console.WriteLine(context.Variables["history"]);

In [34]:
// Using plugins from the repo
using Microsoft.SemanticKernel.AI.ChatCompletion;

var currentDirectory = System.IO.Directory.GetCurrentDirectory();
var pluginsDirectory = Path.Combine(currentDirectory, "Plugins");

var summarizePluginFunctions = kernel.ImportSemanticFunctionsFromDirectory(pluginsDirectory, "SummarizePlugin");

In [35]:
using System.Linq;

var summarizeFunction = summarizePluginFunctions["Summarize"];
var requestSettings = new OpenAIRequestSettings { ServiceId = gpt4ServiceId };
summarizeFunction.ModelSettings.Append(requestSettings);

var pipeline = new []
{
    functions["Chat"],
    summarizePluginFunctions["Summarize"],
    functions["Translate"]
};

In [36]:
await Chat("C# or Python?");

Console.WriteLine(result.GetValue<string>());

Пользователь: Привет
Предложения: Привет! Чем я могу помочь вам сегодня?
